# 01 - ADC Capture & Spectrum Analysis


In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = (14, 5)
rcParams['figure.dpi'] = 100

from osmium.utils.constants import (
    ADC_SAMPLE_RATE, ADC_HW_DECIMATION, DDC_OUTPUT_RATE,
    ATSC_CHANNEL_BW, CHANNEL_TABLE, channel_center_freq,
)
from osmium.capture.adc_capture import ADCCapture

print(f'ADC sample rate: {ADC_SAMPLE_RATE/1e6:.1f} MSPS')
print(f'HW decimation: {ADC_HW_DECIMATION}x')
print(f'DDC output rate: {DDC_OUTPUT_RATE/1e6:.1f} MSPS')

## 1. Select ATSC Channel


In [ ]:

CHANNEL = 36  # ATSC Channel Select 
center_freq = channel_center_freq(CHANNEL)
ch_info = CHANNEL_TABLE[CHANNEL]

print(f'Channel {CHANNEL}:')
print(f'  Lower edge:  {ch_info["lower_edge_hz"]/1e6:.3f} MHz')
print(f'  Center freq: {center_freq/1e6:.3f} MHz')
print(f'  Pilot freq:  {ch_info["pilot_hz"]/1e6:.6f} MHz')
print(f'  Upper edge:  {ch_info["upper_edge_hz"]/1e6:.3f} MHz')

## 2. Load Overlay & Configure ADC

Load the PYNQ base overlay and configure the ADC tile for direct RF sampling.

In [ ]:
try:
    import pynq
    overlay = pynq.Overlay('base.bit')
    print('Base overlay loaded successfully.')
    
    capture = ADCCapture(overlay=overlay, tile_id=0, block_id=0)
    config = capture.configure_tile(channel=CHANNEL)
    
    print(f'\nADC configured:')
    for k, v in config.items():
        print(f'  {k}: {v}')

except ImportError:
    print('PYNQ not available - running in offline mode.')
    print('Load a previously captured IQ file instead (see Section 5).')
    capture = ADCCapture()

## 3. Capture IQ Samples

Perform a DMA burst capture. The default captures ~1M samples (~1.6 ms at 614.4 MSPS).

In [ ]:
NUM_SAMPLES = 2**20  # ~1M samples

try:
    iq_samples = capture.capture_burst(num_samples=NUM_SAMPLES)
    sample_rate = DDC_OUTPUT_RATE
    print(f'Captured {len(iq_samples)} complex samples')
    print(f'Duration: {len(iq_samples)/sample_rate*1e3:.2f} ms')
    print(f'Sample rate: {sample_rate/1e6:.1f} MSPS')

except RuntimeError:
    print('DMA not available. Generating synthetic test signal...')
    # Generate a synthetic ATSC-like signal for offline testing
    sample_rate = DDC_OUTPUT_RATE
    t = np.arange(NUM_SAMPLES) / sample_rate
    # Simulate: pilot tone + band-limited noise (rough ATSC approximation)
    pilot_offset = 309440.559  # Hz - pilot relative to suppressed carrier
    iq_samples = (
        0.1 * np.exp(2j * np.pi * pilot_offset * t)  # pilot
        + 0.5 * (np.random.randn(NUM_SAMPLES) + 1j * np.random.randn(NUM_SAMPLES))  # data
    ).astype(np.complex64)
    print(f'Generated {len(iq_samples)} synthetic samples')

## 4. Spectrum Analysis

Compute and plot the power spectral density to verify the ATSC signal shape.

In [ ]:
from scipy.signal import welch

nperseg = min(4096, len(iq_samples) // 4)
freqs, psd = welch(iq_samples, fs=sample_rate, nperseg=nperseg,
                   return_onesided=False, scaling='density')

sort_idx = np.argsort(freqs)
freqs = freqs[sort_idx]
psd = psd[sort_idx]

psd_db = 10 * np.log10(psd + 1e-20)

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

axes[0].plot(freqs / 1e6, psd_db)
axes[0].set_xlabel('Frequency (MHz from center)')
axes[0].set_ylabel('PSD (dB/Hz)')
axes[0].set_title(f'Full Baseband Spectrum - Channel {CHANNEL}')
axes[0].grid(True, alpha=0.3)
axes[0].axvline(x=-3, color='r', linestyle='--', alpha=0.5, label='-3 MHz')
axes[0].axvline(x=3, color='r', linestyle='--', alpha=0.5, label='+3 MHz')
axes[0].legend()

mask = (freqs > -4e6) & (freqs < 4e6)
axes[1].plot(freqs[mask] / 1e6, psd_db[mask])
axes[1].set_xlabel('Frequency (MHz from center)')
axes[1].set_ylabel('PSD (dB/Hz)')
axes[1].set_title(f'ATSC Channel {CHANNEL} Detail')
axes[1].grid(True, alpha=0.3)
# Mark pilot position 
axes[1].axvline(x=0.309, color='g', linestyle='--', alpha=0.7, label='Pilot (~309 kHz)')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 6))

t_plot = np.arange(min(5000, len(iq_samples))) / sample_rate * 1e6  # us
n_plot = len(t_plot)

axes[0].plot(t_plot, np.real(iq_samples[:n_plot]), linewidth=0.5)
axes[0].set_ylabel('I (In-phase)')
axes[0].set_title('Time Domain - First 5000 Samples')
axes[0].grid(True, alpha=0.3)

axes[1].plot(t_plot, np.imag(iq_samples[:n_plot]), linewidth=0.5, color='orange')
axes[1].set_xlabel('Time (us)')
axes[1].set_ylabel('Q (Quadrature)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Signal statistics
print(f'Signal statistics:')
print(f'  Mean power: {np.mean(np.abs(iq_samples)**2):.4f}')
print(f'  Peak amplitude: {np.max(np.abs(iq_samples)):.4f}')
print(f'  I DC offset: {np.mean(np.real(iq_samples)):.6f}')
print(f'  Q DC offset: {np.mean(np.imag(iq_samples)):.6f}')

## 5. Save / Load Captures


In [ ]:
# Save capture
SAVE_PATH = f'../tests/fixtures/capture_ch{CHANNEL}.npz'

ADCCapture.save_to_file(
    iq_samples, SAVE_PATH,
    metadata={
        'channel': CHANNEL,
        'center_freq_hz': center_freq,
        'sample_rate': sample_rate,
    }
)
print(f'Saved {len(iq_samples)} samples to {SAVE_PATH}')

## 6. Channel Power & SNR Estimate

Rough estimate of in-channel power vs. out-of-channel noise.

In [ ]:
# In-channel: -3 to +3 MHz
in_band = (np.abs(freqs) < 3e6)
out_band = (np.abs(freqs) > 4e6) & (np.abs(freqs) < sample_rate/2 * 0.9)

in_power = np.mean(psd[in_band])
out_power = np.mean(psd[out_band]) if np.any(out_band) else 1e-20

snr_est = 10 * np.log10(in_power / out_power)

print(f'In-band power:  {10*np.log10(in_power+1e-20):.1f} dB/Hz')
print(f'Out-band power: {10*np.log10(out_power+1e-20):.1f} dB/Hz')
print(f'Estimated SNR:  {snr_est:.1f} dB')
print()
if snr_est > 15:
    print('SNR looks good for ATSC demodulation (need ~15 dB for 8VSB).')
elif snr_est > 10:
    print('SNR is marginal. Demod may work but expect RS corrections.')
else:
    print('SNR is low. Check antenna, cable, and channel selection.')